# Notebook for match final results predictions

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle as pk

#pd.set_option('display.max_rows', )
pd.set_option('display.max_columns', 200)

In [2]:
bm = 'PS'
H = bm + 'H'
D = bm + 'D'
A = bm + 'A'

k = 50.0

def get_best(s):

    h = s.FTHG
    a = s.FTAG

    BH = s[H + '_P']
    BD = s[D + '_P']
    BA = s[A + '_P']


    if BH == max(BH, BD, BA, k) and h > a or \
       BD == max(BH, BD, BA, k) and h == a or \
       BA == max(BH, BD, BA, k) and h < a :
        
        return 1
    
    return 0

In [3]:
filename_1 = "Serie-A-2122.csv"
filename_2 = "Serie-A-2223.csv"
filename_3 = "Premier-league-2122.csv"
filename_4 = "Premier-league-2223.csv"
filename_5 = "Bundesliga-2122.csv"
filename_6 = "Bundesliga-2223.csv"
filename_7 = "Liga-2122.csv"
filename_8 = "Liga-2223.csv"
filename_9  = "Ligue1-2122.csv"
filename_10 = "Ligue1-2223.csv"
filename_11 = "Super-lig-2122.csv"
filename_12 = "Super-lig-2223.csv"
filename_13 = "Eredivise-2122.csv"
filename_14 = "Eredivise-2223.csv"
filename_15 = "Liga-1-2122.csv"
filename_16 = "Liga-1-2223.csv"
filename_17 = "Serie-A-2021.csv"

In [4]:
#Loading the datasets

filenames = [filename_1, filename_2, #filename_3, filename_4,
             #filename_5, filename_6, filename_7,
             #filename_8, filename_9, filename_10,
             #filename_11, filename_12, filename_13,
             #filename_14, filename_15, filename_16,
             filename_17]

tmp_features = ['HomeTeam','AwayTeam','FTHG','FTAG', H, D, A]

df = pd.DataFrame()
for name in filenames:
    tmp = pd.read_csv(name, sep=',')
    df = pd.concat([df, tmp[tmp_features]])
df.shape

(1090, 7)

Create the result_index

In [5]:
df[H + '_P'] = round(100/(1 + df[H]*(1/df[D] + 1/df[A])), 2)
df[A + '_P'] = round(100/(1 + df[A]*(1/df[H] + 1/df[D])), 2)
df[D + '_P'] = round(100/(1 + df[D]*(1/df[H] + 1/df[A])), 2)

features = [H, D, A]
target   = [bm + '_income_index']

df[bm + '_income_index'] = df.apply(get_best, axis=1)
df[bm + '_income_index'].value_counts()

PS_income_index
0    693
1    397
Name: count, dtype: int64

In [6]:
df[df[bm + '_income_index'] == 1][[bm + 'H', bm + 'D', bm + 'A']].apply(lambda x: min(x[bm + 'H'], x[bm + 'D'], x[bm + 'A']), axis=1).mean()

1.5545340050377834

In [7]:
df.dropna(inplace=True)

Da notare che è piu importante il valore della precision sugli 1.

### Data preparation

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X = np.concatenate((df[features].to_numpy(), df[features].to_numpy()))
y = np.concatenate((df[target].to_numpy(),   df[target].to_numpy())).squeeze()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=900)

### KNN

In [32]:
from sklearn.neighbors import KNeighborsClassifier

# Create Decision Tree classifer object
clf = KNeighborsClassifier(algorithm='auto', n_jobs=-1, n_neighbors=30, weights='distance')

# Train Decision Tree Classifer
clf = clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.99      0.96      0.98       272
           1       0.94      0.99      0.96       163

    accuracy                           0.97       435
   macro avg       0.97      0.98      0.97       435
weighted avg       0.97      0.97      0.97       435



Salvo il modello

In [33]:
import pickle as pk

# Create Decision Tree classifer object
clf = KNeighborsClassifier(algorithm='auto', n_jobs=-1, n_neighbors=30, weights='distance')

# Train Decision Tree Classifer
clf = clf.fit(X, y)

pk.dump(clf, open("knn-model.sav", 'wb'))

### Random Forest

In [97]:
# Modelling
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(max_depth=8, n_estimators=600, criterion='gini')
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.98      0.80      0.88      1389
           1       0.72      0.97      0.82       720

    accuracy                           0.86      2109
   macro avg       0.85      0.88      0.85      2109
weighted avg       0.89      0.86      0.86      2109



Salvo il modello

In [83]:
import pickle as pk

rf = RandomForestClassifier(max_depth=8, n_estimators=600, criterion='gini')

# Train Decision Tree Classifer
rf = rf.fit(X, y)

pk.dump(clf, open("random_forest-model.sav", 'wb'))

## Support vector machine

In [9]:
from sklearn.svm import SVC

svm = SVC(C=0.0001, degree=1, kernel='poly')
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.66      1.00      0.79      1389
           1       1.00      0.00      0.00       720

    accuracy                           0.66      2109
   macro avg       0.83      0.50      0.40      2109
weighted avg       0.78      0.66      0.52      2109



In [131]:
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 50], 
              'gamma': ['scale', 'auto'],
              'kernel': ['rbf', 'linear', 'poly'],
              'degree': [i for i in range(1, 4)]


}

# definizione del metodo di ricerca migliore combinazione di iper-parametri
grid_search = GridSearchCV(SVC(), param_grid=param_grid, cv=2, n_jobs=8, verbose=True, scoring='precision')

grid_search.fit(X_test, y_test)

Fitting 2 folds for each of 108 candidates, totalling 216 fits


/home/lorenzo/miniconda3/envs/datas/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lorenzo/miniconda3/envs/datas/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lorenzo/miniconda3/envs/datas/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lorenzo/miniconda3/envs/datas/lib/python3.10/s

KeyboardInterrupt: 

In [10]:
# stampa dei migliori parametri e punteggi ottenuti
print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))

NameError: name 'grid_search' is not defined

In [24]:
import pickle as pk

svm = SVC(C=0.0004, degree=1, kernel='poly')
svm.fit(X, y)

pk.dump(svm, open("svm-model.sav", 'wb'))